# Notebook 03 — FMV Modeling (Stages 3 & 4)

## Verizon Cell Tower Lease Fair Market Value Estimation

This notebook implements the **supervised ML model training** and **Over/Under/At-Market classification** pipeline.

### Pipeline Overview

| Stage | Component | Description |
|-------|-----------|-------------|
| **3A** | XGBoost Regressor | Point-estimate FMV via gradient boosting with spatial GroupKFold CV |
| **3B** | Quantile Regression Forest | Prediction intervals at [Q10, Q25, Q50, Q75, Q90] |
| **3C** | Ridge Meta-Learner | Ensembles XGB + QRF-median OOF predictions for final FMV |
| **4** | Classification | Assign OVERPAYING / AT_MARKET / UNDERPAYING using QRF quantile bands |

### Key Design Decisions

1. **Training on CORE sites only** — pseudo-labeled sites whose current rent closely approximates cluster-median FMV. This avoids training on distorted leases.
2. **Spatial GroupKFold** with `state` as the blocking variable — prevents spatial data leakage where nearby sites appear in both train and validation.
3. **Two-model ensemble** — XGBoost captures complex non-linear interactions; QRF provides uncertainty quantification. Ridge meta-learner calibrates the blend.
4. **QRF-based classification** — no arbitrary thresholds. A site is OVERPAYING if its rent exceeds the model’s 75th percentile prediction, UNDERPAYING if below the 25th.

### Inputs
- `../data/processed/clustered_pseudolabeled.parquet` (preferred — output of Notebook 02)
- `../data/synthetic/synthetic_5k_sites.parquet` (fallback — raw synthetic data)

### Outputs
- `../data/processed/fmv_predictions.parquet` — full results with predictions, classifications, and confidence scores

In [ ]:
# ============================================================
# 1  Imports & Configuration
# ============================================================
import numpy as np
import pandas as pd
from scipy import stats
from scipy.stats import median_abs_deviation
import matplotlib.pyplot as plt
import seaborn as sns
import os
import warnings
warnings.filterwarnings('ignore')

from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import GroupKFold
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import (
    mean_absolute_error, mean_squared_error, r2_score,
    confusion_matrix, classification_report, accuracy_score,
    silhouette_score,
)

from xgboost import XGBRegressor

# Quantile forest -- graceful fallback
try:
    from quantile_forest import RandomForestQuantileRegressor
    HAS_QF = True
    print('\u2713 quantile-forest available -- using RandomForestQuantileRegressor')
except ImportError:
    HAS_QF = False
    print('\u2717 quantile-forest not installed -- falling back to sklearn RF with manual quantile estimation')

# HDBSCAN -- needed only for fallback inline clustering
try:
    from sklearn.cluster import HDBSCAN as SklearnHDBSCAN
    HAS_SKLEARN_HDBSCAN = True
    print('\u2713 Using sklearn.cluster.HDBSCAN')
except ImportError:
    HAS_SKLEARN_HDBSCAN = False
    try:
        import hdbscan
        print('\u2713 Using hdbscan library')
    except ImportError:
        print('\u2717 No HDBSCAN available -- inline clustering fallback will use KMeans')

np.random.seed(42)
sns.set_style('whitegrid')
plt.rcParams['figure.dpi'] = 110
plt.rcParams['figure.facecolor'] = 'white'

print('\nAll imports loaded successfully.')

---
## 1. Setup & Data Load

We attempt to load the output of Notebook 02 (`clustered_pseudolabeled.parquet`).
If that file is unavailable, we fall back to the raw synthetic data and run quick
inline clustering + pseudo-labeling so the rest of the notebook can proceed.

The inline fallback replicates the essential logic from Notebook 02:
1. Standardise a small set of clustering features
2. Run HDBSCAN (or KMeans if unavailable)
3. Assign pseudo-labels via cluster-level MAD z-scores

In [ ]:
# ============================================================
# 1.1  Load data (with fallback)
# ============================================================
PRIMARY_PATH   = os.path.join('..', 'data', 'processed', 'clustered_pseudolabeled.parquet')
FALLBACK_PATH  = os.path.join('..', 'data', 'synthetic', 'synthetic_5k_sites.parquet')

if os.path.exists(PRIMARY_PATH):
    df = pd.read_parquet(PRIMARY_PATH)
    print(f'\u2713 Loaded clustered + pseudo-labeled data: {PRIMARY_PATH}')
    print(f'  Shape: {df.shape}')
    NEEDS_INLINE_PREP = False
elif os.path.exists(FALLBACK_PATH):
    df = pd.read_parquet(FALLBACK_PATH)
    print(f'\u2717 Primary data not found. Loaded fallback: {FALLBACK_PATH}')
    print(f'  Shape: {df.shape}')
    NEEDS_INLINE_PREP = True
else:
    raise FileNotFoundError(
        'Neither clustered_pseudolabeled.parquet nor synthetic_5k_sites.parquet found.\n'
        'Please run Notebook 02 or Simulation 00 first.'
    )

print(f'\nColumns ({len(df.columns)}): {list(df.columns[:15])} ...')
df.head(3)

In [ ]:
# ============================================================
# 1.2  Inline clustering + pseudo-labeling (runs only if needed)
# ============================================================
if NEEDS_INLINE_PREP:
    print('Running inline clustering + pseudo-labeling on fallback data...\n')

    # --- Features for clustering (subset of market features) ---
    cluster_features = [
        'census_population_density', 'census_median_income',
        'census_median_home_value', 'underlying_land_value_psf',
        'urban_rural_class', 'fcc_tower_count_5km',
        'tenant_count', 'current_monthly_rent',
    ]
    X_clust = df[cluster_features].copy()
    scaler_clust = StandardScaler()
    X_clust_scaled = scaler_clust.fit_transform(X_clust)

    # --- Clustering ---
    try:
        if HAS_SKLEARN_HDBSCAN:
            clusterer = SklearnHDBSCAN(min_cluster_size=20, min_samples=5)
        else:
            clusterer = hdbscan.HDBSCAN(min_cluster_size=20, min_samples=5)
        labels = clusterer.fit_predict(X_clust_scaled)
        algo_name = 'HDBSCAN'
    except Exception:
        from sklearn.cluster import KMeans
        clusterer = KMeans(n_clusters=15, random_state=42, n_init=10)
        labels = clusterer.fit_predict(X_clust_scaled)
        algo_name = 'KMeans(k=15)'

    df['cluster_id'] = labels
    n_clusters = len(set(labels) - {-1})
    n_noise = (labels == -1).sum()
    print(f'  Clustering: {algo_name} -> {n_clusters} clusters, {n_noise} noise points')

    # --- Pseudo-labeling via cluster-level MAD ---
    df['pseudo_label'] = 'FRINGE'  # default
    for cid in sorted(df['cluster_id'].unique()):
        if cid == -1:
            df.loc[df['cluster_id'] == cid, 'pseudo_label'] = 'OUTLIER'
            continue
        mask = df['cluster_id'] == cid
        rents = df.loc[mask, 'current_monthly_rent'].values
        med = np.median(rents)
        mad = median_abs_deviation(rents, scale='normal')
        if mad < 1e-6:
            mad = np.std(rents) if np.std(rents) > 0 else 1.0
        z_scores = np.abs(rents - med) / mad

        core_mask   = z_scores <= 1.0
        outlier_mask = z_scores > 2.0
        fringe_mask  = ~core_mask & ~outlier_mask

        idx = df.index[mask]
        df.loc[idx[core_mask],    'pseudo_label'] = 'CORE'
        df.loc[idx[outlier_mask], 'pseudo_label'] = 'OUTLIER'
        df.loc[idx[fringe_mask],  'pseudo_label'] = 'FRINGE'

    label_counts = df['pseudo_label'].value_counts()
    print(f'\n  Pseudo-label distribution:')
    for lbl in ['CORE', 'FRINGE', 'OUTLIER']:
        n = label_counts.get(lbl, 0)
        print(f'    {lbl}: {n} ({n/len(df)*100:.1f}%)')

    print('\n\u2713 Inline prep complete.')
else:
    print('\u2713 Data already has clustering + pseudo-labels. Skipping inline prep.')
    label_counts = df['pseudo_label'].value_counts()
    for lbl in ['CORE', 'FRINGE', 'OUTLIER']:
        n = label_counts.get(lbl, 0)
        print(f'  {lbl}: {n} ({n/len(df)*100:.1f}%)')

---
## 1.3  Define Feature Lists

We separate features into two conceptual groups:

| Group | Count | Description |
|-------|-------|-------------|
| **X_MARKET** | 37 | Property, location, demographics, zoning, competition — features the *property* carries regardless of carrier |
| **Z_NETWORK** | 13 | Carrier/network-specific features (tenant count, technology, coverage criticality) |

The model uses **all 50 features** combined.  The split matters later for SHAP decomposition:
we want to quantify how much of FMV is property-driven vs. network-driven.

Three additional categorical columns (`property_type`, `structure_type`, `zoning_district_type`)
are label-encoded and appended, bringing the total to ~53 encoded features.

In [ ]:
# ============================================================
# 1.3  Feature lists
# ============================================================

# 37 property/market/location features
X_MARKET = [
    'census_population_density', 'census_population_3km', 'census_median_income',
    'census_median_home_value', 'census_median_rent', 'census_vacancy_rate',
    'urban_rural_class', 'underlying_land_value_psf', 'hud_safmr',
    'property_tax_rate', 'topography_class', 'terrain_elevation_variance_m',
    'poi_density_1km', 'building_density_500m', 'distance_to_highway_km',
    'ground_elevation_ft', 'unemployment_rate_local',
    'fcc_tower_count_5km', 'fcc_tower_density_sqkm', 'competitor_tower_distance_km',
    'tower_per_capita', 'scarcity_index', 'landlord_concentration',
    'composite_hazard_score',
    'tower_height_ft', 'lease_area_sqft', 'parcel_acreage', 'construction_age',
    'site_access_ordinal', 'permit_approval_difficulty', 'wrluri_index',
    'colocation_required', 'height_restriction_ft',
    'has_building', 'is_ground_tower', 'has_parcel', 'has_zoning_data',
]

# 13 network/carrier features
Z_NETWORK = [
    'tenant_count', 'primary_technology', 'coverage_critical',
    'capacity_utilization_pct', 'high_traffic', 'backhaul_type',
    'has_fiber_connection', 'network_substitutability_score',
    'search_ring_alternative_count', 'inter_site_distance_km',
    'antenna_count', 'structure_capacity_remaining', 'site_priority_tier',
]

# Categoricals that need encoding
CATEGORICALS = ['property_type', 'structure_type', 'zoning_district_type']

print(f'X_MARKET features:  {len(X_MARKET)}')
print(f'Z_NETWORK features: {len(Z_NETWORK)}')
print(f'Total raw features: {len(X_MARKET) + len(Z_NETWORK)}')
print(f'Categoricals to encode: {CATEGORICALS}')

In [ ]:
# ============================================================
# 1.4  Encode categoricals & build final feature matrix
# ============================================================
df_model = df.copy()

# Label-encode each categorical (XGBoost handles ordinal encoding well)
label_encoders = {}
encoded_cat_cols = []
for col in CATEGORICALS:
    if col in df_model.columns:
        le = LabelEncoder()
        df_model[f'{col}_enc'] = le.fit_transform(df_model[col].astype(str))
        label_encoders[col] = le
        encoded_cat_cols.append(f'{col}_enc')
        print(f'  Encoded {col}: {len(le.classes_)} classes -> {col}_enc')

# Build the full encoded feature list
ALL_FEATURES_ENC = X_MARKET + Z_NETWORK + encoded_cat_cols

# Verify all features exist in the DataFrame
missing = [f for f in ALL_FEATURES_ENC if f not in df_model.columns]
if missing:
    print(f'\n\u26a0 WARNING: Missing features ({len(missing)}): {missing}')
    ALL_FEATURES_ENC = [f for f in ALL_FEATURES_ENC if f in df_model.columns]

print(f'\nFinal encoded feature count: {len(ALL_FEATURES_ENC)}')
print(f'Features: {ALL_FEATURES_ENC[:10]} ... (+ {len(ALL_FEATURES_ENC)-10} more)')

---
## Stage 3A: XGBoost Regressor with Spatial Cross-Validation

### Design Choices

1. **Training set**: CORE sites only — these are the pseudo-labeled sites whose current rent
   is close to the cluster median and therefore approximates true FMV.
2. **Target**: `current_monthly_rent` — which for CORE sites *is* the pseudo-FMV.
3. **Spatial CV**: We use `GroupKFold` with `state` as the grouping variable.  This prevents
   data leakage from spatially adjacent sites appearing in both train and validation.
4. **Hyper-parameters**: From `model_config.yaml` — tuned conservatively for stability.

The XGBoost regressor is the primary point-estimate model.  We collect **out-of-fold (OOF)**
predictions for every CORE site — these are needed later for the meta-learner (Stage 3C).

In [ ]:
# ============================================================
# 3A.1  Prepare CORE training set
# ============================================================
core_mask = df_model['pseudo_label'] == 'CORE'
df_core = df_model[core_mask].copy().reset_index(drop=True)

X_core = df_core[ALL_FEATURES_ENC].values.astype(np.float32)
y_core = df_core['current_monthly_rent'].values.astype(np.float32)
groups_core = df_core['state'].values  # spatial grouping

print(f'CORE sites for training: {len(df_core)}')
print(f'Feature matrix shape: {X_core.shape}')
print(f'Target range: ${y_core.min():,.0f} - ${y_core.max():,.0f}')
print(f'Target mean:  ${y_core.mean():,.0f}')
print(f'Unique states (spatial blocks): {len(np.unique(groups_core))}')

In [ ]:
# ============================================================
# 3A.2  GroupKFold Spatial Cross-Validation
# ============================================================
N_SPLITS = 5
gkf = GroupKFold(n_splits=N_SPLITS)

# XGBoost hyper-parameters (from model_config.yaml)
xgb_params = dict(
    n_estimators=500,
    learning_rate=0.05,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    min_child_weight=5,
    reg_alpha=0.1,
    reg_lambda=1.0,
    random_state=42,
    n_jobs=-1,
    verbosity=0,
)

# Storage for OOF predictions and fold metrics
oof_xgb_preds = np.full(len(df_core), np.nan)
fold_metrics = []

print(f'Running {N_SPLITS}-fold GroupKFold spatial CV...')
print(f'XGBoost params: n_estimators={xgb_params["n_estimators"]}, '
      f'lr={xgb_params["learning_rate"]}, max_depth={xgb_params["max_depth"]}\n')

for fold_i, (train_idx, val_idx) in enumerate(gkf.split(X_core, y_core, groups=groups_core)):
    X_tr, X_val = X_core[train_idx], X_core[val_idx]
    y_tr, y_val = y_core[train_idx], y_core[val_idx]

    # Identify held-out states for reporting
    held_out_states = sorted(set(groups_core[val_idx]))

    # Train
    model_xgb = XGBRegressor(**xgb_params)
    model_xgb.fit(
        X_tr, y_tr,
        eval_set=[(X_val, y_val)],
        verbose=False,
    )

    # Predict OOF
    preds_val = model_xgb.predict(X_val)
    oof_xgb_preds[val_idx] = preds_val

    # Fold metrics
    mae  = mean_absolute_error(y_val, preds_val)
    mape = np.mean(np.abs(y_val - preds_val) / np.maximum(y_val, 1)) * 100
    rmse = np.sqrt(mean_squared_error(y_val, preds_val))
    r2   = r2_score(y_val, preds_val)

    fold_metrics.append({
        'fold': fold_i + 1, 'MAE': mae, 'MAPE': mape, 'RMSE': rmse, 'R2': r2,
        'n_train': len(train_idx), 'n_val': len(val_idx),
        'held_out_states': ', '.join(held_out_states[:5]),
    })

    print(f'  Fold {fold_i+1}: MAE=${mae:,.0f}  MAPE={mape:.1f}%  RMSE=${rmse:,.0f}  '
          f'R\u00b2={r2:.3f}  (val={len(val_idx)}, states: {", ".join(held_out_states[:3])}...)')

# Aggregate metrics
metrics_df = pd.DataFrame(fold_metrics)
print(f'\n{"="*65}')
print(f'Spatial CV Summary (CORE sites, {N_SPLITS} folds):')
print(f'  Mean MAE:  ${metrics_df["MAE"].mean():,.0f}  \u00b1 ${metrics_df["MAE"].std():,.0f}')
print(f'  Mean MAPE: {metrics_df["MAPE"].mean():.1f}%  \u00b1 {metrics_df["MAPE"].std():.1f}%')
print(f'  Mean RMSE: ${metrics_df["RMSE"].mean():,.0f}  \u00b1 ${metrics_df["RMSE"].std():,.0f}')
print(f'  Mean R\u00b2:  {metrics_df["R2"].mean():.3f}  \u00b1 {metrics_df["R2"].std():.3f}')
print(f'{"="*65}')

In [ ]:
# ============================================================
# 3A.3  Feature Importance (XGBoost gain-based)
# ============================================================

# Retrain final XGBoost on ALL core data for feature importance & downstream use
xgb_final = XGBRegressor(**xgb_params)
xgb_final.fit(X_core, y_core, verbose=False)

# Extract gain-based importance
importances = xgb_final.feature_importances_
feat_imp_df = pd.DataFrame({
    'feature': ALL_FEATURES_ENC,
    'importance': importances,
}).sort_values('importance', ascending=False).reset_index(drop=True)

# Identify domain for coloring
x_market_set = set(X_MARKET)
z_network_set = set(Z_NETWORK)
feat_imp_df['domain'] = feat_imp_df['feature'].apply(
    lambda f: 'X_MARKET' if f in x_market_set else
              ('Z_NETWORK' if f in z_network_set else 'ENCODED_CAT')
)

print('Top 20 features by XGBoost gain:\n')
print(feat_imp_df.head(20).to_string(index=False))

In [ ]:
# ============================================================
# 3A.4  Top 20 Feature Importance Bar Chart
# ============================================================
top20 = feat_imp_df.head(20).copy()

palette = {'X_MARKET': '#2196F3', 'Z_NETWORK': '#FF9800', 'ENCODED_CAT': '#9C27B0'}
colors = [palette.get(d, '#888') for d in top20['domain']]

fig, ax = plt.subplots(figsize=(10, 8))
bars = ax.barh(
    range(len(top20)-1, -1, -1),
    top20['importance'].values,
    color=colors,
    edgecolor='white', linewidth=0.5,
)
ax.set_yticks(range(len(top20)-1, -1, -1))
ax.set_yticklabels(top20['feature'].values, fontsize=9)
ax.set_xlabel('XGBoost Gain Importance', fontsize=11)
ax.set_title('Stage 3A: Top 20 Features by XGBoost Gain', fontsize=13, fontweight='bold')

# Legend
from matplotlib.patches import Patch
legend_elements = [
    Patch(facecolor='#2196F3', label='X_MARKET (property/location)'),
    Patch(facecolor='#FF9800', label='Z_NETWORK (carrier/network)'),
    Patch(facecolor='#9C27B0', label='Encoded categorical'),
]
ax.legend(handles=legend_elements, loc='lower right', fontsize=9)

plt.tight_layout()
plt.show()

# Print domain breakdown
domain_totals = feat_imp_df.groupby('domain')['importance'].sum()
total_imp = domain_totals.sum()
print('\nImportance by feature domain:')
for d in ['X_MARKET', 'Z_NETWORK', 'ENCODED_CAT']:
    if d in domain_totals.index:
        print(f'  {d}: {domain_totals[d]/total_imp*100:.1f}%')

---
## Stage 3B: Quantile Regression Forest

The QRF provides **prediction intervals** rather than just point estimates.  We predict
quantiles \[0.10, 0.25, 0.50, 0.75, 0.90\] for every site.

- **Q25–Q75** band defines the **at-market range** (Stage 4 classification)
- **Q10–Q90** band defines an outer confidence envelope
- Interval width indicates model uncertainty for each site

If the `quantile-forest` library is not installed, we fall back to a standard sklearn
`RandomForestRegressor` and compute quantiles manually from per-tree predictions.  This is
less accurate (it estimates conditional quantiles from bootstrap distribution rather than
true leaf quantiles) but is a reasonable approximation.

In [ ]:
# ============================================================
# 3B.1  Train QRF on CORE sites
# ============================================================
QUANTILES = [0.10, 0.25, 0.50, 0.75, 0.90]

# Prepare full dataset feature matrix for prediction
X_all = df_model[ALL_FEATURES_ENC].values.astype(np.float32)

if HAS_QF:
    # Use dedicated quantile-forest library
    qrf = RandomForestQuantileRegressor(
        n_estimators=500,
        min_samples_leaf=5,
        random_state=42,
        n_jobs=-1,
    )
    qrf.fit(X_core, y_core)

    # Predict quantiles for ALL sites
    qrf_preds = qrf.predict(X_all, quantiles=QUANTILES)
    # qrf_preds shape: (n_samples, n_quantiles)
    for i, q in enumerate(QUANTILES):
        df_model[f'qrf_q{int(q*100):02d}'] = qrf_preds[:, i]

    print(f'\u2713 QRF trained with quantile-forest library')
    print(f'  Trees: 500, min_samples_leaf: 5')
else:
    # Fallback: train sklearn RF, extract per-tree predictions, compute quantiles manually
    print('Using sklearn RF fallback with manual quantile estimation...')
    rf = RandomForestRegressor(
        n_estimators=500,
        min_samples_leaf=5,
        random_state=42,
        n_jobs=-1,
    )
    rf.fit(X_core, y_core)

    # Collect predictions from all individual trees
    all_tree_preds = np.array([tree.predict(X_all) for tree in rf.estimators_])  # (n_trees, n_samples)

    # Compute quantiles across trees
    for q in QUANTILES:
        df_model[f'qrf_q{int(q*100):02d}'] = np.percentile(all_tree_preds, q * 100, axis=0)

    print(f'\u2713 Sklearn RF fallback -- quantiles computed from {len(rf.estimators_)} trees')

# Preview
qrf_cols = [f'qrf_q{int(q*100):02d}' for q in QUANTILES]
print(f'\nQRF quantile predictions (first 5 sites):')
print(df_model[['site_id', 'current_monthly_rent'] + qrf_cols].head().to_string(index=False))

In [ ]:
# ============================================================
# 3B.2  Prediction Interval Width Distribution
# ============================================================
df_model['pi_width_iqr'] = df_model['qrf_q75'] - df_model['qrf_q25']  # IQR band
df_model['pi_width_90']  = df_model['qrf_q90'] - df_model['qrf_q10']  # 80% band

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# IQR width
axes[0].hist(df_model['pi_width_iqr'], bins=60, color='#2196F3', edgecolor='white', alpha=0.85)
axes[0].axvline(df_model['pi_width_iqr'].median(), color='red', ls='--', lw=1.5,
                label=f'Median: ${df_model["pi_width_iqr"].median():,.0f}')
axes[0].set_xlabel('Prediction Interval Width (Q75 - Q25)', fontsize=10)
axes[0].set_ylabel('Count', fontsize=10)
axes[0].set_title('IQR Band Width (At-Market Range)', fontsize=12, fontweight='bold')
axes[0].legend(fontsize=9)

# 80% band width
axes[1].hist(df_model['pi_width_90'], bins=60, color='#FF9800', edgecolor='white', alpha=0.85)
axes[1].axvline(df_model['pi_width_90'].median(), color='red', ls='--', lw=1.5,
                label=f'Median: ${df_model["pi_width_90"].median():,.0f}')
axes[1].set_xlabel('Prediction Interval Width (Q90 - Q10)', fontsize=10)
axes[1].set_ylabel('Count', fontsize=10)
axes[1].set_title('80% Band Width (Outer Envelope)', fontsize=12, fontweight='bold')
axes[1].legend(fontsize=9)

plt.suptitle('Stage 3B: QRF Prediction Interval Widths', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

print(f'IQR width (Q75-Q25):  median=${df_model["pi_width_iqr"].median():,.0f}, '
      f'mean=${df_model["pi_width_iqr"].mean():,.0f}')
print(f'80% width (Q90-Q10):  median=${df_model["pi_width_90"].median():,.0f}, '
      f'mean=${df_model["pi_width_90"].mean():,.0f}')

---
## Stage 3C: Meta-Learner (Ridge Ensemble)

The meta-learner combines the XGBoost point estimate with the QRF median (Q50) via a
simple Ridge regression trained on **out-of-fold** predictions.  This:

- **Reduces individual model bias** — XGBoost and QRF make different errors
- **Provides a single calibrated FMV estimate** — the final number we report
- **Is interpretable** — a linear combination of two models with transparent weights

We first need QRF OOF predictions (via spatial CV), then combine with XGBoost OOF.

In [ ]:
# ============================================================
# 3C.1  Build OOF meta-features for CORE sites
# ============================================================

# XGBoost OOF predictions already available from 3A spatial CV (oof_xgb_preds)
# Now run QRF spatial CV for unbiased OOF meta-learner training

oof_qrf_q50 = np.full(len(df_core), np.nan)

print('Running QRF spatial CV for OOF meta-features...')
for fold_i, (train_idx, val_idx) in enumerate(gkf.split(X_core, y_core, groups=groups_core)):
    X_tr, X_val = X_core[train_idx], X_core[val_idx]
    y_tr = y_core[train_idx]

    if HAS_QF:
        qrf_fold = RandomForestQuantileRegressor(
            n_estimators=300, min_samples_leaf=5, random_state=42, n_jobs=-1
        )
        qrf_fold.fit(X_tr, y_tr)
        oof_qrf_q50[val_idx] = qrf_fold.predict(X_val, quantiles=[0.50]).ravel()
    else:
        rf_fold = RandomForestRegressor(
            n_estimators=300, min_samples_leaf=5, random_state=42, n_jobs=-1
        )
        rf_fold.fit(X_tr, y_tr)
        tree_preds = np.array([t.predict(X_val) for t in rf_fold.estimators_])
        oof_qrf_q50[val_idx] = np.median(tree_preds, axis=0)

    print(f'  Fold {fold_i+1}: QRF OOF computed for {len(val_idx)} validation sites')

# Check for any NaN gaps
valid_mask = ~np.isnan(oof_xgb_preds) & ~np.isnan(oof_qrf_q50)
print(f'\nValid OOF predictions: {valid_mask.sum()} / {len(df_core)}')

In [ ]:
# ============================================================
# 3C.2  Train Ridge meta-learner on OOF predictions
# ============================================================

# Meta-feature matrix: [XGB_oof, QRF_oof]
meta_X = np.column_stack([oof_xgb_preds[valid_mask], oof_qrf_q50[valid_mask]])
meta_y = y_core[valid_mask]

meta_learner = Ridge(alpha=1.0)
meta_learner.fit(meta_X, meta_y)

print(f'Ridge meta-learner coefficients:')
print(f'  XGBoost weight: {meta_learner.coef_[0]:.4f}')
print(f'  QRF-Q50 weight: {meta_learner.coef_[1]:.4f}')
print(f'  Intercept:      {meta_learner.intercept_:.2f}')

# OOF meta predictions (for diagnostics)
meta_oof_preds = meta_learner.predict(meta_X)
meta_oof_mae  = mean_absolute_error(meta_y, meta_oof_preds)
meta_oof_mape = np.mean(np.abs(meta_y - meta_oof_preds) / np.maximum(meta_y, 1)) * 100
meta_oof_r2   = r2_score(meta_y, meta_oof_preds)

print(f'\nMeta-learner OOF performance (CORE sites):')
print(f'  MAE:  ${meta_oof_mae:,.0f}')
print(f'  MAPE: {meta_oof_mape:.1f}%')
print(f'  R\u00b2:  {meta_oof_r2:.4f}')

In [ ]:
# ============================================================
# 3C.3  Predict FMV for ALL 5,000 sites
# ============================================================

# Full-dataset predictions from each base model (already trained on all CORE data)
xgb_all_preds = xgb_final.predict(X_all)
qrf_q50_all   = df_model['qrf_q50'].values

# Ensemble via meta-learner
meta_X_all = np.column_stack([xgb_all_preds, qrf_q50_all])
df_model['predicted_fmv'] = meta_learner.predict(meta_X_all)

# Store component predictions
df_model['xgb_pred'] = xgb_all_preds

# Clip to plausible range (rent should be positive and bounded)
df_model['predicted_fmv'] = df_model['predicted_fmv'].clip(lower=100, upper=15000)

print(f'FMV predictions generated for all {len(df_model)} sites')
print(f'\nPredicted FMV distribution:')
print(f'  Min:    ${df_model["predicted_fmv"].min():,.0f}/mo')
print(f'  Q25:    ${df_model["predicted_fmv"].quantile(0.25):,.0f}/mo')
print(f'  Median: ${df_model["predicted_fmv"].median():,.0f}/mo')
print(f'  Q75:    ${df_model["predicted_fmv"].quantile(0.75):,.0f}/mo')
print(f'  Max:    ${df_model["predicted_fmv"].max():,.0f}/mo')

In [ ]:
# ============================================================
# 3C.4  Scatter: Predicted FMV vs Current Rent (by pseudo-label)
# ============================================================
fig, ax = plt.subplots(figsize=(9, 8))

label_colors = {'CORE': '#4CAF50', 'FRINGE': '#FFC107', 'OUTLIER': '#F44336'}
label_order  = ['OUTLIER', 'FRINGE', 'CORE']  # draw CORE on top

for lbl in label_order:
    mask = df_model['pseudo_label'] == lbl
    ax.scatter(
        df_model.loc[mask, 'current_monthly_rent'],
        df_model.loc[mask, 'predicted_fmv'],
        alpha=0.35, s=8, c=label_colors[lbl], label=lbl,
    )

# 45-degree reference line
lim = max(df_model['current_monthly_rent'].max(), df_model['predicted_fmv'].max())
ax.plot([0, lim], [0, lim], 'k--', lw=1, alpha=0.6, label='y = x (fair market)')

ax.set_xlabel('Current Monthly Rent ($)', fontsize=11)
ax.set_ylabel('Predicted FMV ($)', fontsize=11)
ax.set_title('Stage 3C: Predicted FMV vs Current Rent\n(colored by pseudo-label)',
             fontsize=13, fontweight='bold')
ax.legend(fontsize=9, loc='upper left')
ax.set_xlim(0, None)
ax.set_ylim(0, None)

plt.tight_layout()
plt.show()

# Correlation stats
r_all = df_model['predicted_fmv'].corr(df_model['current_monthly_rent'])
r_core = df_model.loc[df_model['pseudo_label']=='CORE', 'predicted_fmv'].corr(
    df_model.loc[df_model['pseudo_label']=='CORE', 'current_monthly_rent'])
print(f'Correlation (predicted_fmv, current_rent): all={r_all:.3f}, CORE={r_core:.3f}')

---
## Stage 4: Over/Under/At-Market Classification

We classify each site by comparing its **current rent** against the QRF prediction bands:

| Classification | Rule | Interpretation |
|---------------|------|----------------|
| **OVERPAYING** | `current_rent > QRF Q75` | Carrier is paying above the 75th percentile of estimated FMV |
| **UNDERPAYING** | `current_rent < QRF Q25` | Carrier is paying below the 25th percentile — landlord may seek increase |
| **AT_MARKET** | `QRF Q25 \u2264 current_rent \u2264 QRF Q75` | Rent is within the interquartile range of estimated FMV |

We also compute a **confidence score** = 1 - (IQR / predicted_FMV).  Narrower intervals
(relative to FMV) mean higher confidence in the classification.  A site with predicted FMV
of $2,000/mo and IQR of $200 has confidence 0.90; one with IQR of $1,500 has confidence 0.25.

In [ ]:
# ============================================================
# 4.1  Classify sites
# ============================================================
df_model['model_class'] = np.where(
    df_model['current_monthly_rent'] > df_model['qrf_q75'], 'OVERPAYING',
    np.where(
        df_model['current_monthly_rent'] < df_model['qrf_q25'], 'UNDERPAYING',
        'AT_MARKET'
    )
)

# Confidence: 1 - (IQR / predicted_FMV)
# Narrower interval relative to FMV -> higher confidence
df_model['confidence'] = np.clip(
    1.0 - (df_model['pi_width_iqr'] / df_model['predicted_fmv'].clip(lower=1)),
    0.0, 1.0
)

# Deviation from FMV
df_model['deviation_pct'] = (
    (df_model['current_monthly_rent'] - df_model['predicted_fmv'])
    / df_model['predicted_fmv'].clip(lower=1)
)

# Print classification distribution
class_counts = df_model['model_class'].value_counts()
print('Stage 4: Classification Results')
print('=' * 45)
for cls in ['OVERPAYING', 'AT_MARKET', 'UNDERPAYING']:
    n = class_counts.get(cls, 0)
    pct = n / len(df_model) * 100
    print(f'  {cls:13s}: {n:5d} ({pct:5.1f}%)')

print(f'\nConfidence score: mean={df_model["confidence"].mean():.3f}, '
      f'median={df_model["confidence"].median():.3f}')
print(f'Deviation pct:    mean={df_model["deviation_pct"].mean()*100:.1f}%, '
      f'std={df_model["deviation_pct"].std()*100:.1f}%')

In [ ]:
# ============================================================
# 4.2  Classification Distribution Plots
# ============================================================
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# --- Bar chart of classifications ---
class_order = ['UNDERPAYING', 'AT_MARKET', 'OVERPAYING']
class_colors = ['#2196F3', '#4CAF50', '#F44336']
counts = [class_counts.get(c, 0) for c in class_order]
pcts = [c / len(df_model) * 100 for c in counts]

bars = axes[0].bar(class_order, counts, color=class_colors, edgecolor='white', linewidth=1.5)
for bar, pct in zip(bars, pcts):
    axes[0].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 30,
                 f'{pct:.1f}%', ha='center', va='bottom', fontsize=11, fontweight='bold')
axes[0].set_ylabel('Number of Sites', fontsize=11)
axes[0].set_title('Classification Distribution', fontsize=12, fontweight='bold')

# --- Deviation histogram ---
dev = df_model['deviation_pct'] * 100
axes[1].hist(dev, bins=80, color='#607D8B', edgecolor='white', alpha=0.85)
axes[1].axvline(0, color='red', ls='--', lw=1.5, label='Fair Market (0%)')
axes[1].axvline(dev.median(), color='orange', ls=':', lw=1.5,
                label=f'Median: {dev.median():.1f}%')
axes[1].set_xlabel('Deviation from Predicted FMV (%)', fontsize=10)
axes[1].set_ylabel('Count', fontsize=10)
axes[1].set_title('Rent Deviation Distribution', fontsize=12, fontweight='bold')
axes[1].legend(fontsize=9)

plt.suptitle('Stage 4: Over/Under/At-Market Classification',
             fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

---
## 6. Evaluation Against Ground Truth

Since we have `true_fmv` in our synthetic data, we can compute **true error metrics** and a
**confusion matrix** comparing model classifications against ground-truth classes.

This section would **not exist in production** (where true FMV is unknown) but is essential
for validating the pipeline on synthetic data.  We evaluate:

1. **Regression accuracy**: MAE, MAPE, RMSE, R² of predicted FMV vs true FMV
2. **Classification accuracy**: confusion matrix and classification report (precision, recall, F1) of model class vs true class
3. **Scatter plot**: predicted vs true FMV with 45° line and ±15% bands

In [ ]:
# ============================================================
# 6.1  Regression Accuracy vs Ground Truth
# ============================================================
if 'true_fmv' in df_model.columns:
    y_true = df_model['true_fmv'].values
    y_pred = df_model['predicted_fmv'].values

    gt_mae  = mean_absolute_error(y_true, y_pred)
    gt_mape = np.mean(np.abs(y_true - y_pred) / np.maximum(y_true, 1)) * 100
    gt_rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    gt_r2   = r2_score(y_true, y_pred)
    gt_corr = np.corrcoef(y_true, y_pred)[0, 1]

    # Median absolute error and percentile breakdown
    abs_errors = np.abs(y_true - y_pred)

    print('Evaluation Against Ground Truth (ALL sites)')
    print('=' * 55)
    print(f'  MAE:             ${gt_mae:,.0f}/mo')
    print(f'  Median AE:       ${np.median(abs_errors):,.0f}/mo')
    print(f'  MAPE:            {gt_mape:.1f}%')
    print(f'  RMSE:            ${gt_rmse:,.0f}/mo')
    print(f'  R\u00b2:              {gt_r2:.4f}')
    print(f'  Pearson r:       {gt_corr:.4f}')
    print(f'\n  Error percentiles:')
    for pct in [25, 50, 75, 90, 95]:
        print(f'    P{pct}: ${np.percentile(abs_errors, pct):,.0f}')
else:
    gt_mae = gt_mape = gt_rmse = gt_r2 = gt_corr = None
    print('\u26a0 No true_fmv column found -- skipping ground-truth evaluation.')
    print('  (This is expected for real-world data.)')

In [ ]:
# ============================================================
# 6.2  Classification Confusion Matrix & Report
# ============================================================
if 'true_class' in df_model.columns:
    class_labels = ['UNDERPAYING', 'AT_MARKET', 'OVERPAYING']

    cm = confusion_matrix(
        df_model['true_class'], df_model['model_class'],
        labels=class_labels
    )

    # Confusion matrix heatmap
    fig, ax = plt.subplots(figsize=(7, 6))
    sns.heatmap(
        cm, annot=True, fmt='d', cmap='Blues',
        xticklabels=class_labels, yticklabels=class_labels,
        ax=ax, cbar_kws={'label': 'Count'},
        linewidths=0.5, linecolor='white',
    )
    ax.set_xlabel('Model Prediction', fontsize=11)
    ax.set_ylabel('True Class', fontsize=11)
    ax.set_title('Confusion Matrix: Model vs Ground Truth',
                 fontsize=13, fontweight='bold')
    plt.tight_layout()
    plt.show()

    # Classification report
    print('\nClassification Report:')
    print(classification_report(
        df_model['true_class'], df_model['model_class'],
        labels=class_labels, digits=3
    ))

    acc = accuracy_score(df_model['true_class'], df_model['model_class'])
    print(f'Overall accuracy: {acc:.1%}')
else:
    print('\u26a0 No true_class column -- skipping classification evaluation.')

In [ ]:
# ============================================================
# 6.3  Scatter: Predicted FMV vs True FMV (with 45-degree line)
# ============================================================
if 'true_fmv' in df_model.columns:
    fig, ax = plt.subplots(figsize=(8, 8))

    cls_colors = {'UNDERPAYING': '#2196F3', 'AT_MARKET': '#4CAF50', 'OVERPAYING': '#F44336'}
    for cls in ['UNDERPAYING', 'AT_MARKET', 'OVERPAYING']:
        mask = df_model['model_class'] == cls
        ax.scatter(
            df_model.loc[mask, 'true_fmv'],
            df_model.loc[mask, 'predicted_fmv'],
            alpha=0.3, s=8, c=cls_colors[cls], label=cls,
        )

    # 45-degree perfect prediction line
    lim_max = max(df_model['true_fmv'].max(), df_model['predicted_fmv'].max())
    ax.plot([0, lim_max], [0, lim_max], 'k--', lw=1.5, alpha=0.6, label='Perfect prediction')

    # +/- 15% bands
    xs = np.linspace(0, lim_max, 100)
    ax.fill_between(xs, xs * 0.85, xs * 1.15, alpha=0.08, color='gray', label='\u00b115% band')

    ax.set_xlabel('True FMV ($/mo)', fontsize=11)
    ax.set_ylabel('Predicted FMV ($/mo)', fontsize=11)
    ax.set_title(f'Predicted vs True FMV\n(R\u00b2={gt_r2:.3f}, MAPE={gt_mape:.1f}%)',
                 fontsize=13, fontweight='bold')
    ax.legend(fontsize=9, loc='upper left')
    ax.set_xlim(0, None)
    ax.set_ylim(0, None)

    plt.tight_layout()
    plt.show()
else:
    print('\u26a0 No true_fmv column -- skipping scatter plot.')

---
## 7. Save Outputs

We save the full results DataFrame including:
- All original features and metadata
- QRF quantile predictions (Q10, Q25, Q50, Q75, Q90)
- XGBoost point prediction
- Meta-learner ensemble FMV
- Model classification (OVERPAYING / AT_MARKET / UNDERPAYING)
- Confidence score
- Deviation percentage
- Prediction interval widths

In [ ]:
# ============================================================
# 7.  Save full results
# ============================================================
output_dir = os.path.join('..', 'data', 'processed')
os.makedirs(output_dir, exist_ok=True)

output_path = os.path.join(output_dir, 'fmv_predictions.parquet')

# Select columns to save (all original + model outputs)
model_output_cols = [
    'xgb_pred', 'qrf_q10', 'qrf_q25', 'qrf_q50', 'qrf_q75', 'qrf_q90',
    'predicted_fmv', 'model_class', 'confidence', 'deviation_pct',
    'pi_width_iqr', 'pi_width_90',
]

# Ensure all columns present
save_cols = [c for c in df_model.columns if c not in model_output_cols] + model_output_cols
save_cols = [c for c in save_cols if c in df_model.columns]

df_model[save_cols].to_parquet(output_path, index=False)

file_size_kb = os.path.getsize(output_path) / 1024
print(f'\u2713 Saved results to {output_path}')
print(f'  Shape: {df_model.shape}')
print(f'  File size: {file_size_kb:.0f} KB')
print(f'  Columns saved: {len(save_cols)}')
print(f'\nModel output columns:')
for col in model_output_cols:
    if col in df_model.columns:
        print(f'  {col}: {df_model[col].dtype}')

In [ ]:
# ============================================================
# Final Summary
# ============================================================
print('\n' + '=' * 65)
print('NOTEBOOK 03 SUMMARY -- FMV Modeling (Stages 3 & 4)')
print('=' * 65)

print(f'\n[Stage 3A] XGBoost Spatial CV ({N_SPLITS} folds):')
print(f'  CORE training sites:  {len(df_core)}')
print(f'  Mean MAE:             ${metrics_df["MAE"].mean():,.0f}')
print(f'  Mean MAPE:            {metrics_df["MAPE"].mean():.1f}%')
print(f'  Mean R\u00b2:              {metrics_df["R2"].mean():.3f}')

print(f'\n[Stage 3B] Quantile Regression Forest:')
print(f'  IQR width median:     ${df_model["pi_width_iqr"].median():,.0f}')
print(f'  90% width median:     ${df_model["pi_width_90"].median():,.0f}')
print(f'  QRF library used:     {"quantile-forest" if HAS_QF else "sklearn fallback"}')

print(f'\n[Stage 3C] Meta-Learner Ensemble:')
print(f'  XGB weight:           {meta_learner.coef_[0]:.3f}')
print(f'  QRF weight:           {meta_learner.coef_[1]:.3f}')
print(f'  Intercept:            {meta_learner.intercept_:.1f}')
print(f'  Predicted FMV median: ${df_model["predicted_fmv"].median():,.0f}/mo')

print(f'\n[Stage 4] Classification:')
for cls in ['OVERPAYING', 'AT_MARKET', 'UNDERPAYING']:
    n = class_counts.get(cls, 0)
    print(f'  {cls:13s}: {n:5d} ({n/len(df_model)*100:.1f}%)')
print(f'  Mean confidence:  {df_model["confidence"].mean():.3f}')

if 'true_fmv' in df_model.columns and gt_r2 is not None:
    print(f'\n[Ground Truth Evaluation]:')
    print(f'  MAE:       ${gt_mae:,.0f}')
    print(f'  MAPE:      {gt_mape:.1f}%')
    print(f'  R\u00b2:        {gt_r2:.4f}')
    if 'true_class' in df_model.columns:
        acc = accuracy_score(df_model['true_class'], df_model['model_class'])
        print(f'  Accuracy:  {acc:.1%}')

print(f'\n\u2713 Output saved to: {output_path}')
print(f'\u2713 Ready for Notebook 04 (Validation & Interpretation)')